Now that I've got a good amount of the core code written, let's test the fiducial tracking behavior on the microtubule datasets.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path
import importlib

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV2_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

/home/douglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.describe()

,x [nm],y [nm],z [nm],frame,intensity [photon],offset [photon],loglikelihood,sigma [nm]
count,13484220.000000,13484220.000000,13484220,13484220.000000,13484220.000000,13484220.000000,13484220.000000,13484220.000000
mean,40618.592426,37864.184599,0,20727.189451,3607.341278,311.616173,181.183841,136.937001
std,15795.465154,18613.368523,0,13717.619952,2312.692213,61.775547,550.856857,19.625671
min,20.815000,0.090927,0,100.000000,1.000000,98.010000,-49.945000,54.000000
25%,28866.000000,21163.000000,0,8398.000000,2298.100000,276.150000,91.739000,125.710000
50%,41007.000000,41023.000000,0,20380.000000,3041.100000,297.350000,118.320000,133.640000
75%,53900.000000,53754.000000,0,30837.000000,4251.300000,333.410000,164.520000,143.730000
max,67068.000000,67068.000000,0,49999.000000,85865.000000,4032.700000,267490.000000,378.000000


In [4]:
# Try DataSTORM's cleanup command
clean   = ds.CleanUp()
dfClean = clean(df)

In [5]:
dfClean.describe()

,x [nm],y [nm],z [nm],frame,uncertainty [nm],intensity [photon],offset [photon],loglikelihood,sigma [nm]
count,13431618.000000,13431618.000000,13431618,13431618.000000,1.343162e+07,13431618.000000,13431618.000000,13431618.000000,13431618.000000
mean,40566.593880,37837.341840,0,20728.401176,1.376458e+13,3607.404781,311.518477,178.700684,136.470139
std,15767.119374,18586.255229,0,13716.610325,4.091402e+16,2316.368886,61.786302,549.351270,17.948254
min,20.815000,0.090927,0,100.000000,5.300700e-01,1.000000,98.010000,-49.945000,54.000000
25%,28841.000000,21152.000000,0,8401.000000,4.941400e+00,2295.500000,276.090000,91.641000,125.680000
50%,40964.000000,40988.000000,0,20381.000000,6.515500e+00,3038.900000,297.240000,118.080000,133.580000
75%,53804.000000,53702.000000,0,30835.000000,8.139000e+00,4252.700000,333.300000,163.640000,143.570000
max,67068.000000,67068.000000,0,49999.000000,1.448700e+20,85865.000000,4032.700000,267490.000000,378.000000


## Normal localization processing
Now that the data is cleaned up a bit, we'll proceed with our normal processing.

In [6]:
FilterLLR  = ds.Filter('loglikelihood', '<', 400)
FilterSig1 = ds.Filter('sigma [nm]',    '>', 80)
FilterSig2 = ds.Filter('sigma [nm]',    '<', 200)
df2        = FilterSig2(FilterSig1(FilterLLR(dfClean)))

In [ ]:
df2.describe()

In [7]:
corrector = ds.FiducialDriftCorrect(mergeRadius           = 50,
                                    offTime               = 1,
                                    minSegmentLength      = 20,
                                    minFracFiducialLength = 0.4,
                                    neighborRadius        = 500,
                                    smoothingWindowSize   = 500, #625
                                    smoothingFilterSize   = 300, #500
                                    interactiveSearch     = True)

#searchRegions = [{'xMin' :  2200, 'xMax' :  2800,
#                  'yMin' : 33200, 'yMax' : 33700}]

In [8]:
df3 = corrector(df2)

Frame 49999: 3 trajectories present
2 fiducial(s) detected.


In [9]:
corrector.plotFiducials()

In [ ]:
# Save results
df3.to_csv('FOV_7_noPB_1500mW_10ms_1_MMStack_locResults.dat', index = False)